## Random Forest 

In [179]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
import numpy as np 
import pandas as pd 
from rdkit.Chem.Draw import SimilarityMaps

In [186]:
#Read Features for each drug and combine into featurepairs 
fingerprints = pd.read_csv("/Users/shristi/Documents/InsightDataProject/Data/ProcessedData/Fingerprints.csv")
interactions = pd.read_csv("/Users/shristi/Documents/InsightDataProject/Data/ProcessedData/drug_interactions.csv")


In [194]:
interactions.head()

,Described_interactions
Drug_Pair,
"('Abacavir', 'Lepirudin')",The serum concentration of Abacavir can be dec...
"('Abciximab', 'Lepirudin')",Abciximab may increase the anticoagulant activ...
"('Aceclofenac', 'Lepirudin')",Aceclofenac may increase the anticoagulant act...
"('Acenocoumarol', 'Lepirudin')",Lepirudin may increase the anticoagulant activ...
"('Acetylsalicylic acid', 'Lepirudin')",Acetylsalicylic acid may increase the anticoag...


In [187]:
drug_pairs = interactions['Drug_Pair']

In [188]:
def build_feature_and_label(pair, fingerprint_dictionary, interaction_dictionary):
    drug_1 = pair[0]
    drug_2 = pair[1]
    feature_vector_pair = np.concatenate((fingerprint_dictionary[drug1], fingerprint_dictionary[drug2]), axis = 1)
    label = interaction_dictionary[pair]
    return (feature_vector_pair, label)

In [189]:
drug_pairs = interactions['Drug_Pair']
interactions.set_index('Drug_Pair', inplace=True)
interactions_dict = interactions.to_dict()

drug_pairs = interactions.index
fingerprints = fingerprints[['DrugName', 'Molecular_Fingerprint']]
fingerprints.set_index('DrugName', inplace=True)
fingerprint_dict = fingerprints.to_dict()


In [193]:
features = []
labels = []
print(list(drug_pairs)[2])
for drug_pair in list(drug_pairs): 
    print(drug_pair)
    drug1 = drug_pair.split(',')[0].strip('()').strip("''")
    print(drug1)
    drug2 = (drug_pair.split(',')[1].strip('()')).strip("'").strip("/'")
    print(drug2)
    pair = (drug1, drug2)
    print("pair", pair)
    print(type(pair))
    break
    feature_label = build_feature_and_label(pair, fingerprint_dict, interactions_dict)
    features.append(feature_label[0])
    labels.append(feature_label[1])
    

('Aceclofenac', 'Lepirudin')
('Abacavir', 'Lepirudin')
Abacavir
 'Lepirudin
pair ('Abacavir', " 'Lepirudin")
<class 'tuple'>


In [ ]:
#train the random forest model 
features_train, features_test, label_train, label_test = train_test_split(features, label, test_size=0.25, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=1123)

rf.fit(features, label)

In [ ]:
#use the random forest to predict a new molecule  
m5 = Chem.MolFromSmiles('c1ccccc1O')
fp = numpy.zeros((1,))
DataStructs.ConvertToNumpyArray(AllChem.GetMorganFingerprintAsBitVect(m5, 2), fp)
print(rf.predict((fp,)))
print(rf.predict_proba((fp,)))

In [ ]:
# helper function
def getProba(fp, predictionFunction):
    return predictionFunction((fp,))[0][1]

m5 = Chem.MolFromSmiles('c1ccccc1O')
fig, maxweight = SimilarityMaps.GetSimilarityMapForModel(m5, SimilarityMaps.GetMorganFingerprint, lambda x: getProba(x, rf.predict_proba))